# Install and load required packages

In [ ]:
Pkg.add("Dataframes")
Pkg.add("Optim") # For L-BFGS <https://github.com/JuliaOpt/Optim.jl#basic-api-introduction>

In [1]:
using DataArrays, DataFrames

TODO: Load the original Adult dataset CSV and preprocess it right here
- one-hot encoding

# Load the "Adult" dataset
The Adult dataset is from [here](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/).
I have preprocessed the data. In categorica data missing data is handled as just another category.

In [2]:
#df_data = readtable("data/adult.data", nastrings = ["", "NA", "?"]);
#df_test = readtable("data/adult.test", nastrings = ["", "NA", "?"]);
df_data = readtable("data/adult.processed.data");
df_test = readtable("data/adult.processed.test");
names(df_data)

15-element Array{Symbol,1}:
 :age           
 :workclass     
 :fnlwgt        
 :education     
 :education_num 
 :marital_status
 :occupation    
 :relationship  
 :race          
 :sex           
 :capital_gain  
 :capital_loss  
 :hours_per_week
 :native_country
 :classification

In [3]:
df_test[ 1:3, :age ] # Selecting subset of rows, by column name

3-element DataArrays.DataArray{Int64,1}:
 25
 38
 28

# Set parameters
$K$ will be set to the number of prototypes

In [4]:
# Size or original matrix -1 for target column and -1 for sensitive column
sensitive_column_name = :sex # Sensitive column is "gender" as in the paper
classification_column_name = :classification
K = size(df_data,2) - 2

13

# Auxiliary helper functions

In [89]:
# A nice Unicode named summation function
∑(from::Integer, to::Integer, inner::Function) = sum(inner, colon(from,to))
# Test:
#f(x, k) = x*k
#∑(1, 3, (k) -> f(1, k))

# Partition a matrix two, according to given indices or indicator vector.
# Your matrices need to be column-major, as this is the Julia memory layout.
function partition{T<:Integer}(x::Vector, indices::Vector{T})
    return x[ indices ], x[ setdiff(1:length(X), indices) ]
end
function partition{T<:Integer}(X::Matrix, indices::Vector{T})
    return X[ :, indices ], X[ :, setdiff(1:size(X,2), indices) ]
end
function partition{T<:Integer,U<:Any}(X::SharedArray{U,2}, indices::Vector{T})
    return X[ :, indices ], X[ :, setdiff(1:size(X,2), indices) ]
end
function partition{T<:Bool}(x::Vector, indicator::Vector{T})
    return partition(x, find(indicator))
end
function partition{T<:Bool}(X::Matrix, indicator::Vector{T})
    return partition(X, find(indicator))
end
function partition{T<:Bool,U<:Any}(X::SharedArray{U,2}, indicator::Vector{T})
    return partition(X, find(indicator))
end
# Test:
# @which partition([:first, :second, :third, :fourth], [true, false, true, false])
# @which partition([:first, :second, :third, :fourth], [1,3])

partition (generic function with 8 methods)

# Code for the model

## Definitions
We will differ a bit from the definitions in the paper. Notably in the definition of the random variables $Z$. The definitions we use are: TODO MAKE THESE MATCH THE PAPER, THEN TELL WHAT IS CHANGED
- $\mathbf{X}$ denotes the entire data set, a $(N \times D)$ matrix. The rows of the matrix are the feature vectors $\mathbf{x}_n$ representing attributes of an individual. $\mathbf{X}$ contains neither the classification information (target) column, nor the sensitive column.
- $S$ is a binary random variable representing whether or not a given individual is a member of the "protected set". For the user of the algorithm, this is a decision that is done before running the algorithm.
- $\mathbf{X}_{train}$ denotes the training set.
- $\mathbf{X}_{test}$ denotes the test set.
- $\mathbf{X}^+$ denotes the subset of individuals that are members of the "protected group" i.e. individuals for whom $S=1$. Similarly $\mathbf{X}^-$ denotes the subset of individuals for whom $S=0$.
- Define $\mathbf{X}_{train}^+$, $\mathbf{X}_{train}^-$, $\mathbf{X}_{test}^+$ and $\mathbf{X}_{test}^-$ similarly as above.
- $d$ is a distance measure on $\mathbf{X}$ (e.g. euclidean distance).
- $Z$ is a random integer from the set $\left\{1,\dots,K\right\}$.
- $Y$ is a binary random variable representing the classification decision (we consider binary classification only).

Let $Z$ be a random integer from the set $\left\{1,\dots,K\right\}$. Now we can denote the probability that a datapoint $\mathbf{x}$ maps to a particular prototype $k$ with $\mathbb{P}(Z=k \mid \mathbf{x})$ i.e. given the datapoint $\mathbf{x}$, the probability that $Z$, the index of the prototype, is $k$.

## Definitions in code

From the definitions above, we map some to code and also define additional stuff.

Julia is Column-Majored, so our matrices will be altered accordingly.

- $\mathbf{X}$ is just `𝐗`, but $(D \times N)$ instead of $(N \times D)$.
- $\mathbf{X}_{train}$ is `𝐗train`, $\mathbf{X}_{test}$ is `𝐗test`
- Grouped versions are `𝐗⁺train`, `𝐗⁻train`, `𝐗⁺test`, and `𝐗⁻test` respectively
- $S$ maps to different vectors `S_<someset>` containing the sensitive column for `<someset>`, e.g. `S_𝐗`
- $d$ is defined as a lambda function `d`
- $Z$ is replaced by $\mathbf{Z}$, a matrix of probability vectors $\mathbf{z}$
- $Y$ TODO

Additionally, denote the tuple of prototypes $\mathbf{V} = \left(\mathbf{v}_1,...,\mathbf{v}_K\right)$. Since a single prototype $\mathbf{v}_k$ is a vector of length $D$, $\mathbf{V}$ can be expressed as a ($D \times K$) matrix. This is our optimization variable.

In [6]:
# Symbols: 𝐗 ⁺ ⁻ ∑ 𝐕 𝐱

# Sensitive indices for training and test data
S_𝐗train = convert(Array{Bool}, df_data[ sensitive_column_name ])
S_𝐗test = convert(Array{Bool}, df_test[ sensitive_column_name ])

# Classification vectors for training and test data
𝐲train = convert(Array, df_data[ classification_column_name ])
𝐲test = convert(Array, df_test[ classification_column_name ])

# Drop sensitive and classification columns
idxs_left = setdiff(names(df_data), [sensitive_column_name, classification_column_name])
𝐗train = transpose(convert(Matrix, df_data[idxs_left]))
𝐗test = transpose(convert(Matrix, df_test[idxs_left]))

# Standardize the features with mean 0 variance 1
# TODO: tell why, exponentiation goes quickly out of hand, e^-800 is already NaN on Float64
train_mean = mean(𝐗train,2)[:]
train_var = var(𝐗train,2)[:]
𝐗train = 𝐗train .- train_mean # substraction of vector
𝐗train = 𝐗train ./ train_var # division by vector
𝐗test = 𝐗test .- train_mean # same mean as in training, on purpose
𝐗test = 𝐗test ./ train_var # same variance as in training, on purpose

#𝐗test .- train_mean

# Reconstruct full dataset
𝐗 = hcat(𝐗train, 𝐗test)
S_𝐗 = vcat(S_𝐗train, S_𝐗test)
𝐲 = vcat(𝐲train, 𝐲test)

# Dimensions
D = size(𝐗, 1)
N = size(𝐗, 2)
Ntrain = size(𝐗train, 2)
Ntest = size(𝐗test, 2)

### Distance function
# Lambda
d = (𝐚::Vector, 𝐛::Vector) -> vecnorm(𝐚 - 𝐛) # Euclidean distance
# Non-lambda is slightly faster for calculations, but has to be defined
# for all processes with @everywhere
@everywhere de(𝐚::Vector{Float64}, 𝐛::Vector{Float64}) = vecnorm(𝐚 - 𝐛)

### Optimization variables
# Main optimization variable, matrix holding the prototype vectors.
# Initialized to random Float64 matrix, normal distribution 0 mean 1 variance
𝐕 = randn(D, K)
# Weights or "prototype label predictions" (probabilities)
𝐰 = rand(K) # floats in [0,1)

### Hyperparameters
A = Dict(:z=> 1000, :x=> 0.0001, :y=> 0.1)

print("Done.")

Done.

In [7]:
maximum(𝐗train), minimum(𝐗train), maximum(𝐗test), minimum(𝐗test), maximum(𝐕), minimum(𝐕)

(1.9488022835672973,-5.088141178187719,1.9488022835672973,-5.088141178187719,2.745576743598197,-2.860178737087543)

Divide training and test sets to groups according to whether the individuals are "protected" or not.

In [8]:
(𝐗⁺train, 𝐗⁻train) = partition(𝐗train, S_𝐗train)
(𝐗⁺test, 𝐗⁻test) = partition(𝐗test, S_𝐗test)
"Training:",size(𝐗⁺train), size(𝐗⁻train), "Test:", size(𝐗⁺test), size(𝐗⁻test)

("Training:",(13,21790),(13,10771),"Test:",(13,10860),(13,5421))

## Mapping $\mathbf{X} \rightarrow \mathbf{Z}$
Now we can define a mapping from the original dataset $\mathbf{X}$ to probabilities via the *softmax function*. [Wikipedia](https://en.wikipedia.org/wiki/Softmax_function):
> Softmax function "squashes" a $K$-dimensional vector $\mathbf{z}$ of arbitrary real values to a $K$-dimensional vector $\sigma(\mathbf{z})$ of real values in the range $[0, 1]$ that add up to 1.

Most notably `softmax` returns a probability vector. We will define a modified version that maps a $D$-dimensional vector $\mathbf{x}$ to a $K$-dimensional vector $\sigma(\mathbf{x})$, i.e. the mapping won't necessarily preserve the dimensionality of $\mathbf{x}$.

Also from [Wikipedia](https://en.wikipedia.org/wiki/Multinomial_logistic_regression):
> $$\operatorname{softmax}(k,x_1,\ldots,x_n) = \frac{e^{x_k}}{\sum_{i=1}^n e^{x_i}}$$
> is referred to as the [*softmax function*](https://en.wikipedia.org/wiki/Softmax_function).  The reason is that the effect of exponentiating the values $x_1,\ldots,x_n$ is to exaggerate the differences between them.  As a result, $\operatorname{softmax}(k,x_1,\ldots,x_n)$ will return a value close to 0 whenever $x_k$ is significantly less than the maximum of all the values, and will return a value close to 1 when applied to the maximum value, unless it is extremely close to the next-largest value.  Thus, the softmax function can be used to construct a weighted average that behaves as a smooth function (which can be conveniently differentiated, etc.) and which approximates the [indicator function](https://en.wikipedia.org/wiki/Indicator_function).

So we define, as in the paper equation (2), $$\mathbb{P}(Z=k \mid \mathbf{x}) = \frac{e^{-d(\mathbf{x}, \mathbf{v}_k)}}{\sum_{j=1}^K e^{-d(\mathbf{x}, \mathbf{v}_j)}}$$
where
- $\mathbb{P}(Z=k \mid \mathbf{x})$ is described in [definitions](#Definitions)
- $\mathbf{x}$ is the datapoint
- $\mathbf{v}_k$ is a vector associated with the $k$th prototype
- $d$ is a distance measure between $\mathbf{x}$ and $\mathbf{v}_k$ (e.g. the euclidean distance)

This means that since we have replaced $x_k$ in the softmax with the negative distance between $\mathbf{x}$ and prototype $\mathbf{v}_k$, the softmax returns a value close to 0 whenever the distance from $\mathbf{x}$ to the prototype $\mathbf{v}_k$ is significantly higher than $\min_{j\in\{1,\dots,K\}}\, d(\mathbf{x}, \mathbf{v}_i)$, and close to 1 when applied to the minimum value.

"Mapping from X to Z" in the paper means mapping the vector $\mathbf{x}$ to a probability vector $\mathbf{z}$ of length $K$ via the softmax function. These probability vectors are then used directly for training the classifier.

**In code** this means that $\mathbb{P}(Z=k \mid \mathbf{x})$ is represented by a function taking
- the data point $\mathbf{x}$ which is a `Vector` of length $D$
- $(D \times K)$ `Matrix` of prototypes $\mathbf{V}$ containing all $K$ prototypes $\mathbf{v}_k$ i.e. each prototype is a `Vector` (remember, Column-Major order on matrices)
- a distance measure on $\mathbf{X}$

and returning
- a `Vector` $\mathbf{z}$ of length $K$ representing a [probability vector](https://en.wikipedia.org/wiki/Probability_vector), where each value $z_k$ of the probability vector $\mathbf{z}$ tells how probable it is that $\mathbf{x}$ maps to $\mathbf{v}_k$. Since $\mathbf{z}$ is a probability vector, $\sum_{i=k}^K z_k = 1$.

We will name this function `softmax` and define it as follows:

In [9]:
# TODO: explain, Give in 𝐗, get 𝐙; give in 𝐱, get 𝐳
# TODO: clean up duplicate code

### FOR VECTORS

# This is same as Eq (2) in paper.
function softmax_dist{T<:Number}(𝐱::Vector{T}, 𝐕::Matrix{T}, distanceMeasure::Function)
    K = size(𝐕, 2)
    res = Vector{Float64}(K)
    denominator = Float64(0.0)
    # Use one loop to calculate both numerator and denominator
    for k in 1:K
        res[k] = exp(- distanceMeasure(𝐱, 𝐕[:,k]) )
        denominator += res[k]
    end
    denom = inv(denominator)
    res .* denom
end

function softmax_euclidean{T<:Number}(𝐱::Vector{T}, 𝐕::Matrix{T})
    K = size(𝐕, 2)
    res = Vector{Float64}(K)
    denominator = Float64(0.0)
    # Use one loop to calculate both numerator and denominator
    for k in 1:K
        res[k] = exp(- vecnorm(𝐱 - 𝐕[:,k]) )
        denominator += res[k]
    end
    denom = inv(denominator)
    res .* denom
end

### FOR MATRICES

function softmax_dist{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T}, distanceMeasure::Function)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = Matrix{Float64}(K, N)
    for n in 1:N
        res[:,n] = softmax_dist(𝐗[:,n], 𝐕, distanceMeasure)
    end
    res
end

# Parallel version
function softmax_dist_par{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T}, distanceMeasure::Function)
    nprocs()==CPU_CORES || addprocs(CPU_CORES-1)    
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = SharedArray(Float64, (K, N))
    @sync @parallel for n in 1:N
        for k in 1:K
            res[k,n] = exp(- distanceMeasure(𝐗[:,n], 𝐕[:,k]) )
        end
        res[:,n] = res[:,n] .* inv(sum(res[:,n]))
    end
    res
end

function softmax_euclidean{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T})
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = Matrix{Float64}(K, N)
    for n in 1:N
        denominator = Float64(0.0)
        # Use one loop to calculate both numerator and denominator
        for k in 1:K
            res[k,n] = exp(- vecnorm(𝐗[:,n] - 𝐕[:,k]) )
            denominator += res[k,n]
        end
        res[:,n] = res[:,n] .* inv(denominator)
    end
    res
end

# Parallel version
function softmax_euclidean_par{T<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{T})
    nprocs()==CPU_CORES || addprocs(CPU_CORES-1)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    # Preallocate result matrix, no need to zero it
    res = SharedArray(Float64, (K, N))
    @sync @parallel for n in 1:N
        for k in 1:K
            res[k,n] = exp(- vecnorm(𝐗[:,n] - 𝐕[:,k]) )
        end
        res[:,n] = res[:,n] .* inv(sum(res[:,n]))
    end
    res
end

softmax_euclidean_par (generic function with 1 method)

In [10]:
# For development
𝐙pre = softmax_euclidean_par(𝐗train, 𝐕)
𝐙pre⁺ = softmax_euclidean_par(𝐗⁺train, 𝐕)
𝐙pre⁻ = softmax_euclidean_par(𝐗⁻train, 𝐕);

This approach is akin to using a funky [multinomial logistic regression](https://en.wikipedia.org/wiki/Multinomial_logistic_regression) to "predict the prototype" (category) where a data point $\mathbf{x}$ maps to.
Wikipedia:
> These are all statistical classification problems. They all have in common a dependent variable to be predicted that comes from one of a limited set of items which cannot be meaningfully ordered, as well as a set of independent variables (also known as features, explanators, etc.), which are used to predict the dependent variable. Multinomial logit regression is a particular solution to the classification problem that assumes that a linear combination of the observed features and some problem-specific parameters can be used to determine the probability of each particular outcome of the dependent variable. The best values of the parameters for a given problem are usually determined from some training data.


## Parts of the optimization objective

### $L_z$ &mdash; statistical parity
In code, we will denote $L_z$ with function `Lz`.

In [88]:
### "NAIVE" VERSION FOR UNDERSTANDING THE IMPLEMENTATION

function LzNaive{T<:Number}(𝐗⁺::Matrix{T}, 𝐗⁻::Matrix{T}, 𝐕::Matrix{T}, dist::Function)
    # Operate on matrices and take mean from sample dimension N
    meanp = mean( softmax_dist_par(𝐗⁺, 𝐕, dist), 2 ) # Eq (6)
    meann = mean( softmax_dist_par(𝐗⁻, 𝐕, dist), 2 ) # Similarly for M_k^-
    sum(abs(meanp - meann)) # Eq (7), sum is from k=1 to K
end

### VERSIONS TAKING IN THE DATA SET AND PROTOTYPES
# Optionally a distance measure function can be passed as an argument

function Lz{T<:Number}(𝐗⁺::Matrix{T}, 𝐗⁻::Matrix{T}, 𝐕::Matrix{T})
    return Lz(softmax_euclidean_par(𝐗⁺, 𝐕), softmax_euclidean_par(𝐗⁻, 𝐕))
end

function Lz{T<:Number}(𝐗⁺::Matrix{T}, 𝐗⁻::Matrix{T}, 𝐕::Matrix{T}, dist::Function)
    return Lz(softmax_dist_par(𝐗⁺, 𝐕, dist), softmax_dist_par(𝐗⁻, 𝐕, dist))
end

### VERSION FOR PRECALCULATED 𝐙⁺ and 𝐙⁻
# Note we have only a version for matrices. This is because during performance
# testing I noticed that
#
#   ZZZp = sdata(𝐙shared⁺)
#   ZZZn = sdata(𝐙shared⁻)
#   Lz(ZZZp, ZZZn)
#
# is faster than
#
#   Lz(𝐙shared⁺, 𝐙shared⁻)
#
# So use the Matrix version always and if necessary lift the matrices out
# of the SharedArray with sdata().
#
# TODO: is there way to make a faster parallel version?

function Lz{T<:Number}(𝐙⁺::Matrix{T}, 𝐙⁻::Matrix{T})
    # Operate on matrices and take mean from sample dimension N
    meanp = mean( 𝐙⁺, 2 ) # Eq (6)
    meann = mean( 𝐙⁻, 2 ) # Similarly for M_k^-
    sum(abs(meanp - meann)) # Eq (7), sum is from k=1 to K
end

Lz (generic function with 4 methods)

In [42]:
# Test
LZtrain = Lz(𝐗⁺train, 𝐗⁻train, 𝐕)

0.17883943938377728

### $L_x$ &mdash; information loss
In code, we will denote $L_x$ with function `Lx`.

In [14]:
# Symbols: 𝐗 ⁺ ⁻ ∑ 𝐕 𝐱 𝐲

Note .* elementwise multiplication of softmax_dist() and V, there is no \cdot in the paper in Eq (9), dot product would return a scalar.

In [48]:
### NAIVE VERSION FOR UNDERSTANDING THE IMPLEMENTATION

function LxNaive{T<:Number,U<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{U}, dist::Function)
    D = size(𝐗, 1)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    𝐗hat = zeros(Float64, (D,N))
    sum = Float64(0.0)
    for n in 1:N
        for k in 1:K # Eq (9)
            𝐗hat[:,n] = 𝐗hat[:,n] + (softmax_dist(𝐗[:,n], 𝐕, dist) .* 𝐕[:,k])
        end
        sum += (𝐗[:,n] - 𝐗hat[:,n]) ⋅ (𝐗[:,n] - 𝐗hat[:,n]) # Eq (8)
    end
    return sum, 𝐗hat
end

### VERSIONS TAKING IN THE DATA SET AND PROTOTYPES
# Optionally a distance measure function can be passed as an argument

function Lx{T<:Number,U<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{U})
    return Lx(softmax_euclidean_par(𝐗, 𝐕), 𝐗, 𝐕)
end

function Lx{T<:Number,U<:Number}(𝐗::Matrix{T}, 𝐕::Matrix{U}, dist::Function)
    return Lx(softmax_dist_par(𝐗, 𝐕, dist), 𝐗, 𝐕)
end

### VERSION FOR PRECALCULATED 𝐙

function Lx{T<:Number}(𝐙::SharedArray{T,2}, 𝐗::Matrix{T}, 𝐕::Matrix{T})
    D = size(𝐕, 1)
    K = size(𝐕, 2)
    N = size(𝐙, 2)
    # Keep 𝐙 as SharedArray, will be faster than taking sdata() when fed to the following @parallel loop
    sum = @parallel (+) for n in 1:N # Eq (8)
        𝐱hat_n = zeros(Float64, D)
        for k in 1:K # Eq (9)
            𝐱hat_n = 𝐱hat_n + (𝐙[:,n] .* 𝐕[:,k]) # We are constructing a vector of length D
        end
        (𝐗[:,n] - 𝐱hat_n) ⋅ (𝐗[:,n] - 𝐱hat_n) # "simple squared error"
    end
    return sum
end

# TODO: test if making V into sharedarray increases performance, probably not since V is usually small

Lx (generic function with 3 methods)

In [52]:
# Test
LXtrain = Lx(𝐗train, 𝐕)

182684.39602519403

### $L_y$ &mdash; prediction accuracy
In code, we will denote $L_y$ with function `Ly`.

Essentially here we are letting the optimization pick both the prototypes (i.e. feature vectors) and their predictions (i.e. labels), and the predictions don't have to be discrete 0 and 1, but can be from the range $[0,1]$ and thus themselves can be viewed as probabilities. E.g. let's say that for prototype $v_k$ it's prediction $w_k = 0.82$, then "there is a 82% chance prototype $\mathbf{v}_k$ gets label 1"

In [17]:
### NAIVE VERSION TO HELP UNDERSTAND THE IMPLEMENTATION

function LyNaive{T1<:Number,T2<:Number,T3<:Number}(
        𝐗::Matrix{T1}, 𝐕::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}, dist::Function
    )
    D = size(𝐗, 1)
    N = size(𝐗, 2)
    K = size(𝐕, 2)
    𝐲hat = zeros(Float64, N)
    sum = Float64(0.0)
    # Replace 𝐲hat in Eq (10) with Eq (11), then you get this for loop
    for n in 1:N
        𝐙_n = softmax_dist(𝐗[:,n], 𝐕, dist) # Vector of length K
        for k in 1:K
            𝐲hat[n] = 𝐲hat[n] + (𝐙_n[k] * 𝐰[k])
        end
        # The following line could be replaced with
        # if 𝐲[n] == 1
        #    sum -= log(𝐲hat[n])
        # else # 𝐲[n] == 0
        #    sum -= log(1 - 𝐲hat[n])
        # end
        sum += -𝐲[n] * log(𝐲hat[n])  -  (1 - 𝐲[n]) * log(1 - 𝐲hat[n])
    end
    #return sum, 𝐲hat
    return sum
end

### VERSIONS TAKING IN THE DATA SET AND PROTOTYPES
# Optionally a distance measure function can be passed as an argument

function Ly{T1<:Number,T2<:Number,T3<:Number}(
        𝐗::Matrix{T1}, 𝐕::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}
    )
    # 𝐙 = softmax_euclidean_par(𝐗, 𝐕)
    return Ly(softmax_euclidean_par(𝐗, 𝐕), 𝐲, 𝐰)
end

function Ly{T1<:Number,T2<:Number,T3<:Number}(
        𝐗::Matrix{T1}, 𝐕::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}, dist::Function
    )
    # 𝐙 = softmax_dist_par(𝐗, 𝐕, dist)
    return Ly(softmax_dist_par(𝐗, 𝐕, dist), 𝐲, 𝐰)
end

### VERSION FOR PRECALCULATED 𝐙

# function Ly{T1<:Number,T2<:Number,T3<:Number}(
#         𝐙::Matrix{T1}, 𝐲::Vector{T2}, 𝐰::Vector{T3}
#     )
#     # Copy to shared memory
#     𝐙shared = convert(SharedArray{T1, 2}, 𝐙)
#     return Ly(𝐙shared, 𝐲, 𝐰)
# end

function Ly{T1<:Number,T2<:Number,T3<:Number}(
        𝐙::SharedArray{T1,2}, 𝐲::Vector{T2}, 𝐰::Vector{T3}
    )
    N = size(𝐙, 2)
    # Keep 𝐙 as SharedArray, will be faster than taking sdata() when fed to the following @parallel loop
    sum = @parallel (+) for n in 1:N # Eq (10)
        yhat_n = 𝐙[:,n] ⋅ 𝐰 # Eq (11)
        - 𝐲[n] * log(yhat_n) - (1 - 𝐲[n]) * log(1 - yhat_n)
    end
    return sum
end

Ly (generic function with 4 methods)

In [18]:
# Test
LYtrain = Ly(𝐗train, 𝐕, 𝐲train, 𝐰)

26076.52644269116

## Optimization objective function

In [100]:
# Overall objective function
objective_euclidean(𝐗, 𝐗⁺, 𝐗⁻, 𝐕, 𝐲, 𝐰, A) = A[:z]*Lz(𝐗⁺, 𝐗⁻, 𝐕) + A[:x]*Lx(𝐗, 𝐕) + A[:y]*Ly(𝐗, 𝐕, 𝐲, 𝐰)
objective_dist(𝐗, 𝐗⁺, 𝐗⁻, 𝐕, 𝐲, 𝐰, A, dist) = A[:z]*Lz(𝐗⁺, 𝐗⁻, 𝐕, dist) + A[:x]*Lx(𝐗, 𝐕, dist) + A[:y]*Ly(𝐗, 𝐕, 𝐲, 𝐰, dist)
function objective_pre(𝐗::Matrix, S::Vector{Bool}, 𝐕::Matrix, 𝐲::Vector, 𝐰::Vector, A::Dict)
    # Calculate 𝐙 and partitions once
    𝐙 = softmax_euclidean_par(𝐗, 𝐕)
    (𝐙⁺, 𝐙⁻) = partition(𝐙, S)
    # Use functions that accept precalculated 𝐙
    return A[:z]*Lz(𝐙⁺, 𝐙⁻) + A[:x]*Lx(𝐙, 𝐗, 𝐕) + A[:y]*Ly(𝐙, 𝐲, 𝐰)
end

objective_pre (generic function with 3 methods)

In [93]:
objective_euclidean(𝐗train, 𝐗⁺train, 𝐗⁻train, 𝐕, 𝐲train, 𝐰, A)

117418.72624368257

In [101]:
objective_pre(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A)

117418.72624368257

In [102]:
# @time for i in 1:10
#     objective_euclidean(𝐗train, 𝐗⁺train, 𝐗⁻train, 𝐕, 𝐲train, 𝐰, A)
# end
# @time for i in 1:10
#     objective_pre(𝐗train, S_𝐗train, 𝐕, 𝐲train, 𝐰, A)
# end
#4.119981 seconds (1.24 M allocations: 95.438 MB, 0.42% gc time)
#2.420592 seconds (599.02 k allocations: 87.681 MB, 0.72% gc time)

  2.315172 seconds (596.71 k allocations: 87.516 MB, 0.69% gc time)


In [21]:
objective_euclidean(𝐗test, 𝐗⁺test, 𝐗⁻test, 𝐕, 𝐲test, 𝐰, A)

1489.5333648228848

## Test optimization run

In [ ]:
# Call L-BFGS

# Optimization, running the algorithm
Hyperparameters for the objective function.
In the paper they use grid search to find the parameters. The sets defined here are the same as in the paper.

In [22]:
# Sets of hyperparameters as in paper, for grid search
gridA = Dict(:z => Set([0.1, 0.5, 1.0, 5.0, 10.0]), :x => Set([0, 0.01]), :y => Set([0.1, 0.5, 1.0, 5.0, 10.0]))
# An example of selected hyperparameters, for development
A = Dict(:z => 0.01, :x => 0.5, :y => 1.0)

Dict{Symbol,Float64} with 3 entries:
  :y => 1.0
  :z => 0.01
  :x => 0.5

# TODO
- Overall process with pictures

# Problems/Cons/Notes:
- Let's say that there is a column/feature "Religion" in the dataset.
- Now this paper says we can only say that "Is a member of protected group" or "Is not a member of protected group".
- You have to decide what is the "protected" group, and what is the "normal/non protected" group. You have to decide based on some external criteria who are discriminated against and who are not.
- Let's say we have a dataset with a feature "Religion" and we have 5 different religions represented.
- Now we have to choose which ones are protected and which ones are not.
- The problem of course is that some might be in general discriminated against more than others. There is not necessary even split between the different groups that are discriminated against.

- What we would like to say is that "Religion" is a sensitive feature, and we should not infer _anything_ from it, regardless what it is.

- Does running the algo multiple times help, changing the binary classification each time? Can we extend it so that $S \in {1,...,C}$ where $C$ is the number of categories in the sensitive column.
  - We can extend, just split $L_z$ to multiple cases and the optimization is done to all of them. There will be $c = \frac{(C-1)C}{2} \approx O(C^2)$ pairs. Whether this is computationally still feasible is another question. In the objective function $L_z$ is replaced by $A_{z_1} \cdot L_{z_1} + A_{z_2} \cdot L_{z_2} + \dots + A_{z_c} \cdot L_{z_c}$.

- On the current case where $S \in \left\{0,1\right\}$ once we have set for which rows $S=1$ and $S=0$, we can flip them around without changing anything. This is because we are using statistical parity. This means that from the algorithm's perspective saying that group0 is non-protected and groups 1..4 are protected is the same as saying group1 is protected and other non-protected.